In [ ]:
from sandag_rsm.data_load.zones import load_mgra_data
from sandag_rsm.data_load.triplist import load_trip_list, trip_mode_shares_by_mgra
from sandag_rsm.poi import poi_taz_mgra, attach_poi_taz_skims
from sandag_rsm.zone_agg import aggregate_zones, viewer, viewer2, aggregate_zones_within_districts

In [ ]:
data_dir = "~/Library/CloudStorage/OneDrive-SharedLibraries-CambridgeSystematics/PROJ SANDAG RSM - General/Data"

In [ ]:
mgra = load_mgra_data(data_dir=data_dir, simplify_tolerance=10, topo=True)

In [ ]:
mgra['taz20'] = mgra.taz % 20

In [ ]:
trips = load_trip_list("FromSANDAG-Files/indivTripData_3.csv.gz", data_dir=data_dir)
trip_mode_shares = trip_mode_shares_by_mgra(trips, mgras=mgra.MGRA)

In [ ]:
trip_mode_shares

In [ ]:
mgra = mgra.join(trip_mode_shares.add_prefix("modeshare_"), on='MGRA')

In [ ]:
poi = poi_taz_mgra(mgra)

In [ ]:
poi

In [ ]:
cluster_factors={'popden':1, 'empden':1, 'modeshare_NM':100, 'modeshare_WT':100}

In [ ]:
mgra, cluster_factors = attach_poi_taz_skims(
    mgra,
    "FromSANDAG-Files/traffic_skims_AM.omx",
    names='AM_SOV_TR_M_TIME',
    poi=poi,
    data_dir=data_dir,
    cluster_factors=cluster_factors,
)

In [ ]:
d1 = mgra.query("district27 == 1")

In [ ]:
viewer(d1, color='popden', marker_line_width=1)

In [ ]:
# viewer(d1, color='modeshare_WT', marker_line_width=0)

In [ ]:
# import itertools
# cluster_factors={'popden':1, 'empden':1, 'modeshare_NM':100, 'modeshare_WT':100}
# cluster_factors |= {f"{i}_{j}":1 for i,j in itertools.product(poi.keys(), ['AM_SOV_TR_M_TIME'])}
cluster_factors

In [ ]:
kmeans = aggregate_zones(
    d1, 
    cluster_factors=cluster_factors, 
    n_zones=300,
    explicit_agg=[
        14571, 14574, 14578, 
        [15300, 15303, 15304, 15305],
    ],
    use_xy=1e-6,
)

In [ ]:
from sandag_rsm.zone_agg import aggregate_zones_within_districts

kmeans = aggregate_zones_within_districts(
    mgra, 
    cluster_factors=cluster_factors, 
    n_zones=2000,
    use_xy=1e-6,
    explicit_agg=[
        14571, 14574, 14578, 
        [15300, 15303, 15304, 15305],
    ],
)

In [ ]:
(kmeans)

In [ ]:
kmeans = kmeans.reset_index(drop=True)

In [ ]:
viewer2(edges=kmeans, colors=kmeans, color_col='empden')

In [ ]:
agglom1 = aggregate_zones(d1, cluster_factors=cluster_factors, n_zones=300, method='agglom')

In [ ]:
# viewer2(edges=agglom1, colors=d1, color_col='popden')

In [ ]:
agglom2 = aggregate_zones(
    d1, cluster_factors=cluster_factors, 
    n_zones=300, method='agglom_adj', use_xy=False,
    explicit_agg=[
        14377, 14380, 14384, 
        [15107, 15108, 15106, 15101],
    ],
)

In [ ]:
# viewer2(edges=agglom2, colors=d1, color_col='popden')

In [ ]:
agglom3 = aggregate_zones(
    d1, 
    cluster_factors=cluster_factors, 
    cluster_factors_onehot={'taz':15},
    n_zones=300, 
    method='agglom_adj', 
    use_xy=False,
    explicit_agg=[
        14377, 14380, 14384, 
        [15107, 15108, 15106, 15101],
    ],
)

In [ ]:
# viewer2(edges=agglom3, colors=d1, color_col='popden')

In [ ]:
# viewer2(edges=agglom3, colors=d1, color_col='modeshare_NM')

In [ ]:
# viewer2(edges=agglom3, colors=d1, color_col='taz20')

In [ ]:
agglom3full = aggregate_zones_within_districts(
    mgra, 
    cluster_factors=cluster_factors, 
    cluster_factors_onehot={'taz':100},
    n_zones=2000,
    method='agglom_adj', 
    use_xy=1e-4,
    explicit_agg=[
        14571, 14574, 14578, 
        [15300, 15303, 15304, 15305],
    ],
)

In [ ]:
viewer2(edges=agglom3full, colors=agglom3full, color_col='empden')